In [1]:
from linkedinscraper import LinkedinScraper
import linkedinscraper
from database.database import SessionLocal, engine
from database import crud, database, models
from sqlalchemy import select, func
from database.models import JobPost, Company
import json

In [2]:
db = SessionLocal()

In [3]:
stmt = select(
    JobPost.id,
    JobPost.url,
    JobPost.title,
    JobPost.description,
    Company.name.label('company_name'),
    JobPost.location,
    JobPost.timestamp,
    JobPost.status,
    JobPost.keywords,
    JobPost.search_location
).join(Company).order_by(JobPost.timestamp.desc()).offset(0).limit(3)
results = db.execute(stmt)
for row in results:
    print(row)

(5152, 'https://www.linkedin.com/jobs/view/3652496936/', 'Application Security Engineer', 'Build a Bigger, Better, Bolder FutureImagine working for a company that measures its success based off the growth of its colleagues, a company that i ... (2950 characters truncated) ... fied Ethical Hacker (CEH), Certified Information Systems Security Professional (CISSP), or related certifications with Azure, AWS and GCP are a plus.', 'Little Caesars Pizza', 'Detroit Metropolitan Area', 1695157957, 'success', 'software engineer', 'United States')
(5137, 'https://www.linkedin.com/jobs/view/3724646193/', 'Systems Engineer', 'Overview\n\nSystems Integration Engineer \n\nneeded in our Autocoding business - remote\n\nOur Commitment to YOU!\n\nAt JBT, we value our employees a ... (5052 characters truncated) ... l not be able to give you a status update regarding your application and will not be the individuals making a decision regarding your employment.\n\n', 'JBT Corporation', 'Dallas, TX', 16951571

# filter job posts

doesn't fit will have priority=0
not yet analyzed is -1 or `is None`(default)

In [5]:
res = db.query(JobPost).first()
print(res.priority is None)

False


In [6]:
stmt = select(JobPost, Company).join(Company).order_by(JobPost.timestamp.desc())
res = db.execute(stmt)
for row in res:
    job_post, company = row
    job_post.priority = linkedinscraper.get_priority_of_job_post(job_post, company)
    db.commit()